# Transform logic for Airbnb Reviews

## Initialize Spark sesssion

In [1]:
from pyspark.sql import SparkSession,  DataFrame

spark = SparkSession.builder.appName("clean_reviews").getOrCreate()

24/01/12 20:19:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Read ingested CSV from MinIO

In [2]:
s3_source_path = "s3://datalake/raw/kaggle_airbnb/reviews.csv"

df = spark.read\
        .option("sep",",")\
        .option("inferSchema", "true")\
        .option("header", "true")\
        .option("multiline","true")\
        .option("quote", '"')\
        .option("escape", "\\")\
        .option("escape", '"')\
        .option("encoding", "UTF-8")\
        .option("ignoreLeadingWhiteSpace", "true")\
        .option("ignoreTrailingWhiteSpace", "true")\
        .csv(s3_source_path)

24/01/12 20:19:21 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


## Preview Data

In [3]:
print(f"Total: {df.count()}")

df.limit(5).toPandas().head()

Total: 84849


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


## List reviews with Null comments

In [4]:
df_null_comments = df.filter(df.comments.isNull())
print(f"Null comment count: {df_null_comments.count()}")
df_null_comments.show()

Null comment count: 17


+----------+--------+----------+-----------+-------------+--------+
|listing_id|      id|      date|reviewer_id|reviewer_name|comments|
+----------+--------+----------+-----------+-------------+--------+
|    461567|11614247|2014-04-09|   12120141|        Abbey|    NULL|
|      9460|10563024|2014-02-24|   12498029|        Debra|    NULL|
|    910784| 9950520|2014-01-21|     179481|       Enrico|    NULL|
|     10695|   52378|2010-06-13|     105412|          Wei|    NULL|
|   1018204|10024984|2014-01-26|   10571694|       Jordan|    NULL|
|   6079216|34824019|2015-06-12|   31556342|         Mack|    NULL|
|   3354614|18103248|2014-08-21|   12426758|         Jeff|    NULL|
|   3554558|24863045|2015-01-03|   24488791|      Eleanor|    NULL|
|   1790020|15640556|2014-07-13|   16884291|      Michael|    NULL|
|     23430| 8347394|2013-10-27|    5034901|          Jim|    NULL|
|    774659| 7116754|2013-09-07|    7654662|    Elizabeth|    NULL|
|    585418|10782872|2014-03-07|   11979005|    

## Drop reviews with Null comments

In [5]:
def drop_null_comments(df: DataFrame ) -> DataFrame:
    with_comments=df.filter(df.comments.isNotNull())
    return with_comments

df_clean = df.transform(drop_null_comments)
print(f"Clean record count : {df_clean.count()}")

Clean record count : 84832


## List remaining reviews with Null comments (none)

In [6]:
df_clean.filter(df_clean.comments.isNull()).show()

+----------+---+----+-----------+-------------+--------+
|listing_id| id|date|reviewer_id|reviewer_name|comments|
+----------+---+----+-----------+-------------+--------+
+----------+---+----+-----------+-------------+--------+



In [7]:
spark.stop()